In [8]:
import os
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

import dialogflow

In [9]:
#Настройки 
updater = Updater(token='1467046292:AAH7PvFc9BDku9zin7Ted9U8YM9HZawWSNM') # Токен API к Telegram
dispatcher = updater.dispatcher

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'newagent-whqc-f01baded1e14.json'# скачнный JSON
DIALOGFLOW_PROJECT_ID = 'newagent-whqc' #PROJECT ID из DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
SESSION_ID = 'GBNLP10_2020_bot'  # ID бота из телеграма

In [10]:
def startCommand1(update: Update, context: CallbackContext):
    update.message.reply_text('Добрый день!')
    update.message.reply_text('Я автоматическая система разговора с человеками.')
    update.message.reply_text('Если желаете что либо приобрести напишите: \nкупить: "наименование товара"')

def HelpCommand(update: Update, context: CallbackContext):
    update.message.reply_text('Добрый день!')   
    
def textMessage1(update: Update, context: CallbackContext):
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)
    text_input = dialogflow.types.TextInput(text=update.message.text , language_code=DIALOGFLOW_LANGUAGE_CODE)
    query_input = dialogflow.types.QueryInput(text=text_input)
    
    try:
        response = session_client.detect_intent(session=session, query_input=query_input)
    except InvalidArgument:
         raise

    text = response.query_result.fulfillment_text
    if text:
        update.message.reply_text(response.query_result.fulfillment_text)
    else:
        update.message.reply_text('Что?')

In [11]:
# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", startCommand1))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage1))

In [12]:
updater.start_polling(clean=True)
# Останавливаем бота, если были нажаты Ctrl + C
updater.idle()